## Install Libraries


In [1]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 29.4 MB/s eta 0:00:00


## Import Libraries


In [2]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [4]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [5]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 32
channel_2 = 64
channel_3 = 128
pool_kernel_size = 2

#kernel_size_1 = 3
#kernel_size_2 = 5
#kernel_size_3 = 7

#pad_size_1 = 2
#pad_size_2 = 2
#pad_size_3 = 3

fc_count_1 = 256

class Net(nn.Module):
    def __init__(self, kernel_size_1, kernel_size_2, kernel_size_3, pad_size_1, pad_size_2, pad_size_3):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [6]:
def train_cifar(config):
    net = Net(
        config["kernel_size_1"], 
        config["kernel_size_2"], 
        config["kernel_size_3"], 
        config["pad_size_1"], 
        config["pad_size_2"], 
        config["pad_size_3"]
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-3)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [7]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["kernel_size_1"], 
        best_result.config["kernel_size_2"], 
        best_result.config["kernel_size_3"], 
        best_result.config["pad_size_1"], 
        best_result.config["pad_size_2"], 
        best_result.config["pad_size_3"]
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [8]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"kernel_size_1": tune.grid_search([3, 5, 7]), 
          "kernel_size_2": tune.grid_search([3, 5, 7]), 
          "kernel_size_3": tune.grid_search([3, 5, 7]), 
          "pad_size_1": tune.grid_search([2, 3]), 
          "pad_size_2": tune.grid_search([2, 3]), 
          "pad_size_3": tune.grid_search([2, 3])}



In [9]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


2023-04-08 07:48:23,022	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=2014) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_07-48-21/train_cifar_bd8e1_00000_0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2_2023-04-08_07-48-24/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:13<00:00, 12492137.68it/s]


(train_cifar pid=2014) Extracting /root/ray_results/train_cifar_2023-04-08_07-48-21/train_cifar_bd8e1_00000_0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2_2023-04-08_07-48-24/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_07-48-21/train_cifar_bd8e1_00000_0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2_2023-04-08_07-48-24/data
(train_cifar pid=2014) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_bd8e1_00000,0.6008,2023-04-08_07-49-45,True,,7a4c502b1b4340c4b9d15eea35e074b5,"0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.12698,172.28.0.12,2014,True,77.5524,4.57394,77.5524,1680940185,0,,10,bd8e1_00000,0.00359225
train_cifar_bd8e1_00001,0.6036,2023-04-08_07-50-51,True,,7a4c502b1b4340c4b9d15eea35e074b5,"1_kernel_size_1=5,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.14068,172.28.0.12,2014,True,65.9515,4.58719,65.9515,1680940251,0,,10,bd8e1_00001,0.00359225
train_cifar_bd8e1_00002,0.598,2023-04-08_07-51-55,True,,7a4c502b1b4340c4b9d15eea35e074b5,"2_kernel_size_1=7,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.13733,172.28.0.12,2014,True,63.608,4.55188,63.608,1680940315,0,,10,bd8e1_00002,0.00359225
train_cifar_bd8e1_00003,0.6295,2023-04-08_07-52-58,True,,7a4c502b1b4340c4b9d15eea35e074b5,"3_kernel_size_1=3,kernel_size_2=5,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.05431,172.28.0.12,2014,True,63.4001,4.65081,63.4001,1680940378,0,,10,bd8e1_00003,0.00359225
train_cifar_bd8e1_00004,0.6164,2023-04-08_07-54-03,True,,7a4c502b1b4340c4b9d15eea35e074b5,"4_kernel_size_1=5,kernel_size_2=5,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.08215,172.28.0.12,2014,True,65.1971,4.60209,65.1971,1680940443,0,,10,bd8e1_00004,0.00359225
train_cifar_bd8e1_00005,0.591,2023-04-08_07-55-07,True,,7a4c502b1b4340c4b9d15eea35e074b5,"5_kernel_size_1=7,kernel_size_2=5,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.15304,172.28.0.12,2014,True,64.0306,4.54142,64.0306,1680940507,0,,10,bd8e1_00005,0.00359225
train_cifar_bd8e1_00006,0.6279,2023-04-08_07-56-12,True,,7a4c502b1b4340c4b9d15eea35e074b5,"6_kernel_size_1=3,kernel_size_2=7,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.05298,172.28.0.12,2014,True,64.8643,4.52101,64.8643,1680940572,0,,10,bd8e1_00006,0.00359225
train_cifar_bd8e1_00007,0.6105,2023-04-08_07-57-16,True,,7a4c502b1b4340c4b9d15eea35e074b5,"7_kernel_size_1=5,kernel_size_2=7,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.09532,172.28.0.12,2014,True,64.0169,4.55883,64.0169,1680940636,0,,10,bd8e1_00007,0.00359225
train_cifar_bd8e1_00008,0.6064,2023-04-08_07-58-20,True,,7a4c502b1b4340c4b9d15eea35e074b5,"8_kernel_size_1=7,kernel_size_2=7,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.10767,172.28.0.12,2014,True,63.9956,4.60164,63.9956,1680940700,0,,10,bd8e1_00008,0.00359225
train_cifar_bd8e1_00009,0.6272,2023-04-08_07-59-24,True,,7a4c502b1b4340c4b9d15eea35e074b5,"9_kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=2,pad_size_2=2,pad_size_3=2",084eb1ea0567,10,1.07712,172.28.0.12,2014,True,64.222,4.54664,64.222,1680940764,0,,10,bd8e1_00009,0.00359225


...

(train_cifar pid=2014) Extracting /root/ray_results/train_cifar_2023-04-08_07-48-21/train_cifar_bd8e1_00215_215_kernel_size_1=7,kernel_size_2=7,kernel_size_3=7,pad_size_1=3,pad_size_2=3,pad_size_3=3_2023-04-08_11-42-02/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_07-48-21/train_cifar_bd8e1_00215_215_kernel_size_1=7,kernel_size_2=7,kernel_size_3=7,pad_size_1=3,pad_size_2=3,pad_size_3=3_2023-04-08_11-42-02/data
(train_cifar pid=2014) Files already downloaded and verified
(train_cifar pid=2014) Finished Training


2023-04-08 11:43:08,520	INFO tune.py:798 -- Total run time: 14084.24 seconds (14084.01 seconds for the tuning loop).


Best trial config: {'kernel_size_1': 3, 'kernel_size_2': 7, 'kernel_size_3': 5, 'pad_size_1': 3, 'pad_size_2': 3, 'pad_size_3': 2}
Best trial final validation loss: 0.9910286456156688
Best trial final validation accuracy: 0.6555


100%|██████████| 170498071/170498071 [00:13<00:00, 13106447.55it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.6047


In [10]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
96,0.991029,0.6555,4.696726,False,65.608118,3,7,5,3,3,2
51,0.996654,0.6552,4.593421,False,64.190676,3,7,7,3,2,2
186,0.992422,0.6550,4.754282,False,65.876746,3,7,7,2,3,3
147,1.010641,0.6549,4.744510,False,65.812495,3,5,5,3,2,3
210,0.993324,0.6536,4.787794,False,65.337295,3,5,7,3,3,3
160,1.000783,0.6536,4.619867,False,66.083484,5,7,7,3,2,3
105,0.984212,0.6529,4.685443,False,66.159377,3,7,7,3,3,2
214,1.003207,0.6524,4.802513,False,65.285455,5,7,7,3,3,3
196,1.015816,0.6497,4.819001,False,65.002980,5,7,3,3,3,3
129,1.012253,0.6491,4.688585,False,65.178288,3,5,7,2,2,3


In [11]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
59,1.177717,0.5792,4.479516,False,63.072880,7,5,3,2,3,2
11,1.160255,0.5887,4.661449,False,65.123570,7,3,5,2,2,2
65,1.167523,0.5909,4.709996,False,63.833083,7,3,5,2,3,2
5,1.153044,0.5910,4.541420,False,64.030641,7,5,3,2,2,2
119,1.150256,0.5924,4.745869,False,64.788964,7,3,5,2,2,3
29,1.150538,0.5941,4.591957,False,62.866307,7,3,3,3,2,2
95,1.130681,0.5964,4.606018,False,64.895185,7,5,5,3,3,2
2,1.137326,0.5980,4.551876,False,63.608016,7,3,3,2,2,2
110,1.131510,0.5982,4.520314,False,63.501704,7,3,3,2,2,3
83,1.135607,0.5982,4.624486,False,65.055364,7,3,3,3,3,2
